In [3]:
import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from utility import *

### Data loading

In [4]:
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = to_categorical(np.ravel(y_train))
y_test = to_categorical(np.ravel(y_test))

print(f'x_train shape: {x_train.shape}')
print(f'y_train shape: {y_train.shape}')

label_names = {
    0: 'plane',
    1: 'car',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 10)


In [5]:
def preprocess_images(images_set):
    return images_set * 1. / 255   

# Neural Network

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    #rescale=1./255,
    rotation_range=25,
    horizontal_flip=True,
    vertical_flip=False,
    featurewise_center=True,
    featurewise_std_normalization=True,
    #zca_whitening=True,
    fill_mode='reflect',
    validation_split=0.02
)

train_datagen.fit(x_train)

In [7]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow_addons.layers.maxout import Maxout

def maxout_nn_model(show_summary=False):
    inputs = tf.keras.layers.Input((32, 32, 3))
    
    hidden = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    hidden = tf.keras.layers.MaxPool2D((2,2))(hidden)
    hidden = tf.keras.layers.BatchNormalization()(hidden)
    hidden = Maxout(num_units=16)(hidden)
        
    hidden = tf.keras.layers.Conv2D(48, (3, 3), activation='relu', padding='same')(hidden)
    hidden = tf.keras.layers.MaxPool2D((2,2))(hidden)
    hidden = tf.keras.layers.BatchNormalization()(hidden)
    #hidden = tf.keras.layers.Dropout(0.1)(hidden)
    hidden = Maxout(num_units=24)(hidden)
    
    hidden = tf.keras.layers.Conv2D(96, (3, 3), activation='relu', padding='same')(hidden)
    hidden = tf.keras.layers.MaxPool2D((2,2))(hidden)
    hidden = tf.keras.layers.BatchNormalization()(hidden)
    #hidden = tf.keras.layers.Dropout(0.1)(hidden)
    hidden = Maxout(num_units=48)(hidden)
    
    hidden = tf.keras.layers.Conv2D(192, (4, 4), activation='relu', padding='valid')(hidden)
    hidden = tf.keras.layers.BatchNormalization()(hidden)
    #hidden = tf.keras.layers.Dropout(0.1)(hidden)
    hidden = Maxout(num_units=96)(hidden)
    
    hidden = tf.keras.layers.Flatten()(hidden)
    
    hidden = tf.keras.layers.Dense(256, activation='relu')(hidden)
    hidden = Maxout(num_units=128)(hidden)
    
    outputs = tf.keras.layers.Dense(10, activation='softmax')(hidden)
    
    model = tf.keras.Model(inputs, outputs)
    metrics = [
        tfa.metrics.F1Score(num_classes=10, average="weighted"),
        'accuracy'
    ]
    model.compile(
        optimizer=RMSprop(learning_rate=0.0002),
        loss='categorical_crossentropy',
        metrics=metrics
    )
    
    if show_summary:
        print(model.summary())
    
    return model

In [8]:
model = maxout_nn_model(show_summary=True)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 32)        128       
_________________________________________________________________
maxout (Maxout)              (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 48)        6960      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 48)          0     

D:\SoftwareProjects\anaconda3\envs\tf_new\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

save_best = ModelCheckpoint('models/best_weights_final.hdf5', save_best_only=True, monitor='val_f1_score', mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_f1_score', mode='max', factor=0.2, patience=5, verbose=1, cooldown=3)

history = model.fit(
    train_datagen.flow(x_train, y_train, batch_size=64, subset='training'),
    steps_per_epoch=765,
    validation_data=train_datagen.flow(x_train, y_train, batch_size=64, subset='validation'),
    validation_steps=16,
    callbacks=[save_best, reduce_lr],
    epochs=40
)

Epoch 1/40
765/765 [==============================] - ETA: 0s - loss: 1.5735 - f1_score: 0.4290 - accuracy: 0.4338

D:\SoftwareProjects\anaconda3\envs\tf_new\lib\site-packages\tensorflow\python\keras\metrics.py:254: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  warnings.warn('Metric %s implements a `reset_states()` method; rename it '


765/765 [==============================] - 41s 51ms/step - loss: 1.5735 - f1_score: 0.4290 - accuracy: 0.4338 - val_loss: 1.3245 - val_f1_score: 0.5208 - val_accuracy: 0.5280
Epoch 2/40
765/765 [==============================] - 39s 51ms/step - loss: 1.2646 - f1_score: 0.5442 - accuracy: 0.5479 - val_loss: 1.2085 - val_f1_score: 0.5597 - val_accuracy: 0.5520
Epoch 3/40
765/765 [==============================] - 39s 51ms/step - loss: 1.1368 - f1_score: 0.5895 - accuracy: 0.5921 - val_loss: 1.1103 - val_f1_score: 0.5982 - val_accuracy: 0.6040
Epoch 4/40
765/765 [==============================] - 39s 51ms/step - loss: 1.0566 - f1_score: 0.6216 - accuracy: 0.6236 - val_loss: 1.0073 - val_f1_score: 0.6389 - val_accuracy: 0.6420
Epoch 5/40
765/765 [==============================] - 43s 56ms/step - loss: 0.9911 - f1_score: 0.6471 - accuracy: 0.6485 - val_loss: 1.0064 - val_f1_score: 0.6531 - val_accuracy: 0.6510
Epoch 6/40
765/765 [==============================] - 41s 53ms/step - loss: 0.941

In [10]:
plot_history(history.history)

In [16]:
model.load_weights('models/best_weights_final.hdf5')
model.evaluate(train_datagen.flow(x_test, y_test))

313/313 [==============================] - 8s 26ms/step - loss: 0.7119 - f1_score: 0.7578 - accuracy: 0.7578


[0.7119130492210388, 0.7578243613243103, 0.7577999830245972]